# System

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -U imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.10.1
    Uninstalling imbalanced-learn-0.10.1:
      Successfully uninstalled imbalanced-learn-0.10.1


In [ ]:
!pip install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 43.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
#clean_word
import re
from pythainlp.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#Word_to_vector
from sklearn.feature_extraction.text import TfidfVectorizer
#model
from sklearn.ensemble import RandomForestClassifier
#split
from sklearn.model_selection import train_test_split
#Accuracy
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data preprocessing

In [ ]:
#Load and preprocess the data
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/imbd/data/4_imdb_master.csv',encoding='ISO-8859-1')  # Replace 'shopping-comment.csv' with your dataset
data.shape
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  100000 non-null  int64 
 1   type        100000 non-null  object
 2   review      100000 non-null  object
 3   label       100000 non-null  object
 4   file        100000 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [ ]:
#Choose the colum and remove colum
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.drop(['file'], axis=1, inplace=True)
data.info()
data.head()
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   type    100000 non-null  object
 1   review  100000 non-null  object
 2   label   100000 non-null  object
dtypes: object(3)
memory usage: 2.3+ MB


(100000, 3)

In [ ]:
# Cleaning the text
def cleanText(text):
    text = re.sub('[^A-Za-z]+', ' ', text)
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ',text)
    text = re.sub(r'\s+',' ',text)
    text = re.sub(r'<[^>]+>', ' ',text)
    sentence = word_tokenize(text)
    result = [word for word in sentence if word not in sentence and " " not in word]
    return text

data['review'] = data['review'].apply(cleanText)
data.head()

,type,review,label
0,test,Once again Mr Costner has dragged out movie fo...,neg
1,test,This is an example of why the majority of acti...,neg
2,test,First of all hate those moronic rappers who co...,neg
3,test,Not even the Beatles could write songs everyon...,neg
4,test,Brass pictures movies is not fitting word for ...,neg


In [ ]:
#variables
X = data['review']
y = data['label']

# Random Forest Model

In [ ]:
# Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenization & Vectorization (ในที่นี้ใช้ TF-IDF)
tfidf_vectorizer = TfidfVectorizer(max_features=2000)  # ปรับค่า max_features ตามความเหมาะสม
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Random Forest
rf_model = RandomForestClassifier(n_estimators=100,class_weight='balanced', random_state=42)  # ปรับพารามิเตอร์ต่าง ๆ ตามความเหมาะสม
rf_model.fit(X_train_tfidf, y_train)
y_pred = rf_model.predict(X_test_tfidf)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.50415


# Prediction

In [ ]:
new_text = ["The comments already posted for this film do not do it justice. It is a very low budget sci-fi film made by lesser known people and should be judged that way. To say the film shouldn't be made is plain wrong. How do people learn without doing? Small pictures should be encouraged, not made fun of just because they are not of the highest quality."]
new_text_tfidf = tfidf_vectorizer.transform(new_text)
prediction = rf_model.predict(new_text_tfidf)
print("Predicted class:", prediction[0])

# Displaying Results
for text, prediction_index in zip(new_text,prediction):
    if prediction_index == "pos":
        prediction_label = 1
    elif prediction_index == "unsup":
        prediction_label = 0
    else:
        prediction_label = -1

print("Predicted class:", prediction_label)

Predicted class: unsup
Predicted class: 0


# Pickle

In [ ]:
import pickle
with open('rf_model.pkl', 'wb') as file:
    pickle.dump(rf_model, file)
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)